In [1]:
# === Kepler Light Curves → Features → LightGBM → Adversarial Attack (with tqdm) ===
import os, sys, warnings, json
from pathlib import Path
import numpy as np
import pandas as pd
from astropy.io import fits
import lightkurve as lk
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import lightgbm as lgb
from tqdm.auto import tqdm

# --------------------------
# Config
# --------------------------
DATA_FOLDER = Path("kepler_lightcurves/lightcurves_all/kepler")  # change if needed
MANIFEST_CSV = Path("data/kepler_lightcurves/kepler_local_lightcurves_status.csv")  # your manifest
MAX_FILES = 1000
RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)

# --------------------------
# Helpers: robust feature engineering from a light curve
# --------------------------
def robust_stats(flux):
    f = flux[np.isfinite(flux)]
    if f.size == 0:
        return dict(mean=np.nan, std=np.nan, mad=np.nan, p5=np.nan, p95=np.nan,
                    amp=np.nan, skew=np.nan, kurt=np.nan)
    mean = float(np.mean(f))
    std = float(np.std(f))
    med = float(np.median(f))
    mad = float(np.median(np.abs(f - med)))
    p5, p95 = float(np.percentile(f, 5)), float(np.percentile(f, 95))
    amp = p95 - p5
    if std > 0:
        z = (f - mean) / std
        skew = float(np.mean(z**3))
        kurt = float(np.mean(z**4) - 3.0)  # Fisher
    else:
        skew = 0.0
        kurt = -3.0
    return dict(mean=mean, std=std, mad=mad, p5=p5, p95=p95, amp=amp, skew=skew, kurt=kurt)

def lin_slope(time, flux):
    mask = np.isfinite(time) & np.isfinite(flux)
    t, f = time[mask], flux[mask]
    if t.size < 5:
        return np.nan
    t0 = (t - t.mean()) / (t.std() + 1e-12)
    coeffs = np.polyfit(t0, f, 1)
    return float(coeffs[0])

def lag1_autocorr(x):
    x = x[np.isfinite(x)]
    if x.size < 3:
        return np.nan
    x0, x1 = x[:-1], x[1:]
    s0, s1 = np.std(x0), np.std(x1)
    if s0 == 0 or s1 == 0:
        return 0.0
    return float(np.corrcoef(x0, x1)[0, 1])

def extract_features_from_fits(path: Path):
    """Read a Kepler lightcurve FITS and return a dict of engineered features."""
    try:
        lc_obj = lk.read(path)  # KeplerLightCurve (usually)
        if hasattr(lc_obj, "pdcsap_flux") and lc_obj.pdcsap_flux is not None:
            time = np.asarray(lc_obj.time.value, dtype=float)
            flux = np.asarray(lc_obj.pdcsap_flux.value, dtype=float)
        elif hasattr(lc_obj, "flux") and lc_obj.flux is not None:
            time = np.asarray(lc_obj.time.value, dtype=float)
            flux = np.asarray(lc_obj.flux.value, dtype=float)
        else:
            raise RuntimeError("Unsupported LightCurve object (no flux field).")
    except Exception:
        # Raw FITS fallback
        with fits.open(path, memmap=False) as hdul:
            data = hdul[1].data
            time = np.asarray(data["TIME"], dtype=float)
            if "PDCSAP_FLUX" in data.names:
                flux = np.asarray(data["PDCSAP_FLUX"], dtype=float)
            elif "SAP_FLUX" in data.names:
                flux = np.asarray(data["SAP_FLUX"], dtype=float)
            else:
                raise RuntimeError(f"No PDCSAP_FLUX/SAP_FLUX in {path.name}")

    mask = np.isfinite(time) & np.isfinite(flux)
    if mask.sum() == 0:
        raise RuntimeError(f"No finite flux/time in {path.name}")

    # Keep only valid
    time, flux = time[mask], flux[mask]

    # Relative flux
    med = np.median(flux)
    rel = (flux / (med if med != 0 else 1.0)) - 1.0

    stats = robust_stats(rel)
    n = int(rel.size)
    frac_nan = float(1.0 - (mask.sum() / len(mask))) if len(mask) > 0 else 0.0
    slope = lin_slope(time, rel)
    ac1 = lag1_autocorr(rel)
    rms = float(np.sqrt(np.mean((rel - np.median(rel))**2)))

    feats = dict(
        n_cadences=n,
        frac_nan=frac_nan,
        slope=slope,
        ac1=ac1,
        rms=rms,
        **stats,
    )
    return feats

# --------------------------
# Load manifest (labels) and collect FITS paths
# --------------------------
fits_files = sorted(DATA_FOLDER.glob("**/*.fits"))[:MAX_FILES]
if len(fits_files) == 0:
    raise FileNotFoundError(f"No FITS files found under {DATA_FOLDER.resolve()}")

name_to_label = {}
if MANIFEST_CSV.exists():
    manifest = pd.read_csv(MANIFEST_CSV)
    keycol = "productFilename" if "productFilename" in manifest.columns else ("filename" if "filename" in manifest.columns else None)
    if keycol is None:
        raise ValueError("manifest.csv must have 'productFilename' or 'filename' column.")

    def row_to_label(row):
        # Prefer explicit boolean-ish "is_confirmed"
        ic = row.get("is_confirmed", None)
        if pd.notna(ic):
            s = str(ic).strip().upper()
            if s in ("TRUE", "1", "YES", "Y"):
                return 1
            if s in ("FALSE", "0", "NO", "N"):
                return 0
        # Fall back to disposition/status strings if present
        disp = str(row.get("dispositions", "")).upper()
        stat = str(row.get("status_best", "")).upper()
        if "CONFIRMED" in disp or "CONFIRMED" in stat:
            return 1
        if "FALSE POSITIVE" in disp or "FALSE POSITIVE" in stat:
            return 0
        # default unlabeled → 0
        return 0

    manifest["label"] = manifest.apply(row_to_label, axis=1)
    # Map by just the basename to match f.name
    name_to_label = dict(zip(manifest[keycol].astype(str).map(lambda p: Path(p).name),
                             manifest["label"].astype(int)))
else:
    warnings.warn("manifest.csv not found — defaulting all labels to 0 (not confirmed).")
    name_to_label = {}

# --------------------------
# Build feature table (with tqdm)
# --------------------------
rows, skips = [], 0
for f in tqdm(fits_files, desc="Extracting features", unit="file"):
    try:
        feats = extract_features_from_fits(f)
        label = int(name_to_label.get(f.name, 0))
        feats.update(dict(filename=f.name, label=label))
        rows.append(feats)
    except Exception as e:
        skips += 1
        tqdm.write(f"[SKIP] {f.name}: {e}")

if len(rows) < 2 or len({r['label'] for r in rows}) < 2:
    print("[WARN] Not enough labeled variety; training may be trivial or fail. Check manifest labels.")

df = pd.DataFrame(rows).dropna(axis=1, how="all")[:MAX_FILES]
feature_cols = [c for c in df.columns if c not in ("filename", "label")]
X = df[feature_cols].copy()
# Median impute and cast to float
X = X.fillna(X.median(numeric_only=True)).astype(float).values
y = df["label"].astype(int).values

# --------------------------
# Train / Valid split and LightGBM model
# --------------------------
# Keep test_size reasonable for very small datasets
if len(y) > 10:
    test_size = 0.3
else:
    test_size = 0.5

stratify = y if len(np.unique(y)) > 1 else None
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=test_size, random_state=RANDOM_SEED, stratify=stratify
)

train_ds = lgb.Dataset(X_train, label=y_train)
valid_ds = lgb.Dataset(X_test, label=y_test, reference=train_ds)

params = {
    "objective": "binary",
    "metric": "auc",
    "learning_rate": 0.05,
    "num_leaves": 31,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.9,
    "bagging_freq": 1,
    "min_data_in_leaf": 10,
    "verbose": -1,
    "seed": RANDOM_SEED,
}

booster = lgb.train(
    params, train_ds, num_boost_round=1000,
    valid_sets=[train_ds, valid_ds], valid_names=["train", "valid"],
    callbacks=[lgb.early_stopping(50), lgb.log_evaluation(50)]
)

def predict_label(arr):
    p = booster.predict(arr.reshape(1, -1), num_iteration=booster.best_iteration)[0]
    return int(p >= 0.5), float(p)

# Basic eval
y_pred = (booster.predict(X_test, num_iteration=booster.best_iteration) >= 0.5).astype(int)
print("\n=== Evaluation ===")
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=4))

# --------------------------
# White-box greedy attack (tree-path heuristic) + tqdm over iterations
# --------------------------
def _dump_model(booster):
    return booster.dump_model()

def _traverse_to_leaf(node, x):
    path = []
    nd = node
    while 'leaf_index' not in nd:
        path.append(nd)
        fid = nd['split_feature']
        thr = float(nd['threshold'])
        nd = nd['left_child'] if x[fid] <= thr else nd['right_child']
    path.append(nd)
    return nd, path

def _min_delta_to_flip(threshold, current_value, eps=1e-6):
    # Smallest signed perturbation to cross the threshold
    return (threshold + eps) - current_value if current_value <= threshold else (threshold - eps) - current_value

def greedy_tree_attack(x0, target_label, booster, feature_bounds=None, max_iters=50, eps=1e-6, show_tqdm=True):
    model = _dump_model(booster)
    x = x0.copy().astype(float)
    lo, hi = (None, None) if feature_bounds is None else feature_bounds

    def prob1(z):
        return float(booster.predict(z.reshape(1, -1), num_iteration=booster.best_iteration)[0])
    def current_label(z):
        p = prob1(z)
        return (1 if p >= 0.5 else 0), p

    y_cur, p_cur = current_label(x)
    changes = []
    best_x = x.copy()
    best_p_for_target = p_cur if target_label == 1 else (1.0 - p_cur)

    if y_cur == target_label:
        return x, True, changes

    it_iter = tqdm(range(max_iters), desc="Greedy attack", unit="iter", leave=False) if show_tqdm else range(max_iters)
    for _ in it_iter:
        candidates = []
        for tree in model["tree_info"]:
            root = tree["tree_structure"]
            _, path_nodes = _traverse_to_leaf(root, x)
            for node in path_nodes:
                if 'split_index' not in node:
                    continue
                fidx = node['split_feature']
                thr = float(node['threshold'])
                cur_val = x[fidx]
                delta = _min_delta_to_flip(thr, cur_val, eps=eps)
                if abs(delta) < 1e-12:
                    continue
                new_val = cur_val + delta
                if feature_bounds is not None:
                    if not (lo[fidx] <= new_val <= hi[fidx]):
                        continue
                    new_val = float(np.clip(new_val, lo[fidx], hi[fidx]))
                new_x = x.copy()
                new_x[fidx] = new_val
                _, new_p = current_label(new_x)
                gain_for_target = new_p if target_label == 1 else (1.0 - new_p)
                candidates.append((gain_for_target, abs(delta), fidx, new_val - cur_val, new_x, new_p))

        if not candidates:
            break

        # Prefer best target-probability gain; tie-break by smaller move
        candidates.sort(key=lambda t: (-t[0], t[1]))
        gain, absd, fidx, delta, new_x, new_p = candidates[0]
        x = new_x
        changes.append((fidx, delta))

        y_cur, p_cur = current_label(x)
        score_for_target = p_cur if target_label == 1 else (1.0 - p_cur)
        if score_for_target > best_p_for_target:
            best_p_for_target = score_for_target
            best_x = x.copy()

        if y_cur == target_label:
            return x, True, changes

    final_label, _ = current_label(best_x)
    return best_x, (final_label == target_label), changes

# --------------------------
# Choose a sample and attack
# --------------------------
if len(X_test) == 0:
    raise RuntimeError("Empty test set; cannot run attack.")

neg_indices = np.where(y_test == 0)[0]
if neg_indices.size == 0:
    print("[INFO] No negative samples in test set; flipping a positive to negative instead.")
    idx = 0
    desired = 0
else:
    idx = int(neg_indices[0])
    desired = 1

x0 = X_test[idx].copy()
y0, p0 = predict_label(x0)

# Per-feature bounds from training data (min/max)
lo = np.nanmin(X_train, axis=0)
hi = np.nanmax(X_train, axis=0)
x_adv, success, changes = greedy_tree_attack(
    x0, desired, booster, feature_bounds=(lo, hi), max_iters=200, show_tqdm=True
)

y1, p1 = predict_label(x_adv)

# --------------------------
# Report changes
# --------------------------
print("\n=== Adversarial Attack Result ===")
print(f"Original pred: {y0} (p1={p0:.4f})  →  Desired: {desired}")
print(f"After attack:  {y1} (p1={p1:.4f})  | Success: {success}")

delta = x_adv - x0
changed_idxs = np.where(np.abs(delta) > 1e-12)[0]
if changed_idxs.size == 0:
    print("No feature changed (already at desired class or no viable move).")
else:
    print("\nChanged features (feature_name : delta [percent of (max-min)]):")
    ranges = (hi - lo) + 1e-12
    for fi in changed_idxs:
        fname = feature_cols[fi]
        d = float(delta[fi])
        pct = 100.0 * (d / ranges[fi])
        print(f" - {fname:12s}: {d:+.6g}  [{pct:+.2f}% of range]")

# Context: top importances
imp = booster.feature_importance(importance_type="gain")
imp_pairs = sorted(zip(feature_cols, imp), key=lambda t: -t[1])
print("\nTop feature importances (gain):")
for n, (fname, val) in enumerate(imp_pairs[:10], 1):
    print(f"{n:2d}. {fname:12s} : {val:.3f}")


/home/rayan.banerjee/python-3.13.7/lib/python3.13/site-packages/lightkurve/prf/__init__.py:7: UserWarning: Warning: the tpfmodel submodule is not available without oktopus installed, which requires a current version of autograd. See #1452 for details.
  warnings.warn(


Extracting features:   0%|          | 0/1000 [00:00<?, ?file/s]

Training until validation scores don't improve for 50 rounds
[50]	train's auc: 1	valid's auc: 0.794576
Early stopping, best iteration is:
[12]	train's auc: 0.999776	valid's auc: 0.96339

=== Evaluation ===
Accuracy: 0.9833333333333333
              precision    recall  f1-score   support

           0     0.9833    1.0000    0.9916       295
           1     0.0000    0.0000    0.0000         5

    accuracy                         0.9833       300
   macro avg     0.4917    0.5000    0.4958       300
weighted avg     0.9669    0.9833    0.9751       300



/home/rayan.banerjee/python-3.13.7/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/rayan.banerjee/python-3.13.7/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/rayan.banerjee/python-3.13.7/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.c

Greedy attack:   0%|          | 0/200 [00:00<?, ?iter/s]


=== Adversarial Attack Result ===
Original pred: 0 (p1=0.0102)  →  Desired: 1
After attack:  1 (p1=0.5024)  | Success: True

Changed features (feature_name : delta [percent of (max-min)]):
 - frac_nan    : -0.0158854  [-88.79% of range]
 - ac1         : -0.849478  [-83.60% of range]
 - rms         : -0.00262563  [-9.49% of range]
 - mad         : -0.00198986  [-7.20% of range]
 - p5          : +0.00376532  [+8.54% of range]
 - p95         : -0.00477912  [-12.31% of range]
 - amp         : -0.00856087  [-10.91% of range]

Top feature importances (gain):
 1. ac1          : 208.864
 2. amp          : 143.045
 3. p95          : 102.274
 4. mean         : 74.124
 5. rms          : 69.020
 6. mad          : 65.005
 7. p5           : 64.006
 8. frac_nan     : 14.697
 9. slope        : 13.076
10. kurt         : 3.513
